In [ ]:
!pip install datasets
!pip install transformers
!pip install torchaudio
!pip install jiwer
!pip install soundfile
!pip install librosa
!pip install evaluate
!pip install torchaudio

In [ ]:
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Not connected to a GPU')
# else:
#   print(gpu_info)

In [ ]:
# from numba import cuda 
# device = cuda.get_current_device()
# device.reset()

In [ ]:
from transformers import pipeline
from datasets import load_dataset, load_metric, Audio, ClassLabel, load_from_disk, Features, Value
import evaluate

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!git lfs

In [ ]:
print("----------------- Loading Datasets... -----------------")
features = Features(
    {
        "Unnamed: 0": Value("int32"),
        "Root": Value("string"),
        "SpeakerID": Value("string"),
        "Component": Value("string"),
        "Group": Value("int32"),
        "Age": Value("int32"),
        "Gender": Value("string"),
        "CEF": Value("string"),
        "DialectRegion": Value("string"),
        "Duration (seconds)": Value("float"),
        "Duration (days)": Value("float"),
        "Sentence": Value("string"),
        "Path": Audio(sampling_rate=48_000),
    }
)

# To prepare the csv:
# add column 'audio' with absolut path to the audio files (Excel function: =CONCAT('path/to/audio_files/folder' B2))
test_from_csv = load_dataset('csv', data_files={'test':"/path/file.csv", },)
                                   # data_dir="/path/")
# trainset = DatasetDict()
# file = open("/path/file.json")
# spl = json.load(file)
# train = load_dataset('csv', data_files={'test': "/path/file.csv"})

# trainset['train'] = train["test"]

# dataset = DatasetDict()
# test_set = trainset["train"].select(spl['1']['7'])
test_from_csv = test_from_csv.cast(features)

test_from_csv = test_from_csv.remove_columns(
    ["Unnamed: 0","Root", "DialectRegion", "Age", "SpeakerID", "Group", "Gender", "Duration (seconds)", "Duration (days)", "CEF", "Component"])

test = test_from_csv['test']

test['Path']
print("----------------- Loading Datasets complete. -----------------\n\n")

In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["Sentence"]).lower()
    return batch

test = test.map(remove_special_characters)

In [ ]:
from transformers import pipeline
from datasets import load_dataset, load_metric, Audio, ClassLabel, load_from_disk, Features, Value
import evaluate
import pandas as pd

transcriber = pipeline("automatic-speech-recognition", model='modddddel/whisper-native-children-5-dutch', device=2,
                      generate_kwargs = {"language":"<|nl|>","task": "transcribe"},
                      )

wer = evaluate.load("wer")
labels = []
preds = []
i = 0

for recording in test:
    label = labels.append(recording['Sentence'])
    pred = preds.append(transcriber(recording['Path'])['text'])
    i += 1
    print(str(i) + '/' + str(len(test)))
    print('predicted: ' + preds[i-1])
    print('actual: ' + labels[i-1])

# Recommended: save the results in a CSV file to use later for comparison
# (to avoid having to run the model(s) again)
df = pd.DataFrame({'reference': labels, 'hypothesis': preds})
df.to_csv('/path/file.csv')

print( 100 * wer.compute(predictions=preds, references=labels))

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = '/path/file.csv'
df = pd.read_csv(file_path)

# Apply the regex operation to the 'hypothesis' column
df['hypothesis'] = df['hypothesis'].replace(r'�\s*', '', regex=True)

# Save the modified DataFrame to a new CSV file
output_file_path = "/path/file.csv"
df.to_csv(output_file_path, index=False)


In [ ]:
import jiwer
import pandas as pd

df = pd.read_csv('/path/file.csv')

out = jiwer.process_words(
    df['reference'].values.tolist(),
    df['hypothesis'].values.tolist(),
)

print(jiwer.visualize_alignment(out))

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = '/path/Data/data/meta/csv/nl/training/nc/results/results9_a.csv'
df = pd.read_csv(file_path)

# Apply the regex operation to the 'hypothesis' column
df['hypothesis'] = df['hypothesis'].replace(r'�\s*', '', regex=True)

# Save the modified DataFrame to a new CSV file
output_file_path = "/path/Data/data/meta/csv/nl/training/nc/results/results9_a_modified.csv"
df.to_csv(output_file_path, index=False)


In [ ]:
import jiwer
import pandas as pd

df = pd.read_csv('/path/Data/data/meta/csv/nl/training/nc/results/results9_a_modified.csv')

out = jiwer.process_words(
    df['reference'].values.tolist(),
    df['hypothesis'].values.tolist(),
)

print(jiwer.visualize_alignment(out))